<a href="https://colab.research.google.com/github/Venture-Coding/SUNY-Buffalo-ML-and-self-learning/blob/main/NLP/BERT_based_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment Analysis with BERT

  We will do the following operations to train a sentiment analysis model:
1. Install Transformers library;
2. Load the BERT Classifier and Tokenizer along with Input modules;
3. Download the IMDB Reviews Data and create a processed dataset (this will take several operations;
4. Configure the Loaded BERT model and Train for Fine-tuning
5. Make Predictions with the Fine-tuned Model


In [2]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 596 kB 37.9 MB/s 
     |████████████████████████████████| 895 kB 31.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [4]:
#lets understand the loaded model
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [5]:
import tensorflow as tf
import pandas as pd

Importing the data, tar file, directly from the Stanford repository.

In [6]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

data = tf.keras.utils.get_file(fname = "aclImdb_v1.tar.gz",
                               origin=url,
                               untar=True,
                               cache_dir='.',
                               cache_subdir='')


84140032/84125825 [==============================] - 2s 0us/step


This data supposedly has a proportion of unlabelled reviews. So in this implementation, we shall be removing that part of the data. (Thats the unsup folder inside the train folder)

In [7]:
# The shutil module offers a number of high-level operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(data), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))

['urls_neg.txt', 'neg', 'urls_pos.txt', 'pos', 'urls_unsup.txt', 'labeledBow.feat', 'unsupBow.feat']


In [8]:
## Time for train test split

train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2, 
    subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [9]:
#separate into arrays, transpose, convert to dataframe
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['REVIEW', 'LABEL']
train['REVIEW'] = train['REVIEW'].str.decode("utf-8")
train.head()

,REVIEW,LABEL
0,Canadian director Vincenzo Natali took the art...,1
1,I gave this film 10 not because it is a superb...,1
2,I admit to being somewhat jaded about the movi...,1
3,"For a long time, 'The Menagerie' was my favori...",1
4,A truly frightening film. Feels as if it were ...,0


In [10]:
#SIMILARLY FOR TEST DATA
for i in test.take(1):
  test_feat = i[0].numpy()
  test_lab = i[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['REVIEW', 'LABEL']
test['REVIEW'] = test['REVIEW'].str.decode("utf-8")
test.head()

,REVIEW,LABEL
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


The BERT model needs suitable objects to be considered as inputs. So we will use InputExample to create input sequences from our Pandas data.

In [11]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

Learning from above, we'll create data_to_examples, and examples_to_tf_data functions.

In [12]:
#take input of train and test datasets and convert each row into an InputExample object.

def data_to_examples(train, test,   REVIEW, LABEL): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[REVIEW], 
                                                          text_b = None,
                                                          label = x[LABEL]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[REVIEW], 
                                                          text_b = None,
                                                          label = x[LABEL]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = data_to_examples(train, test, 
                                                                  'REVIEW', 'LABEL')
  
                                                      
  

In [13]:
# Tokenize the InputExample objects, 
# Create the requd. input format with the tokenized objects, 
# Finally, create an input dataset that we can feed to the model.

def examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, hence taking a look at it helps
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,      # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            padding = 'max_length',     # pads to the right by default
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [14]:
train_InputExamples, validation_InputExamples = data_to_examples(train, test, 'REVIEW', 'LABEL')

#Calling the created function on the output of earlier created function, in tandem with the earlier created BERT tokenizer 

train_data = examples_to_tf_dataset(list(train_InputExamples), tokenizer) 
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

Finally we have the processed data in  input sequences ready for our BERT model.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

In [23]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie',
                  'Possibly could have been better. Certain aspects were appreciable though.',
                  'Possibly could have been better. Certain aspects were barely appreciable though.']


In [24]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good : 
 Positive
One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie : 
 Negative
Possibly could have been better. Certain aspects were appreciable though. : 
 Positive
Possibly could have been better. Certain aspects were barely appreciable though. : 
 Negative


 **Success!** Look just how excellent the BERT trains the model in one or two epochs only. A single word, that makes a review good or bad is **correctly** identified, mapped and the review is classified so well, understanding the context.